In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import uuid
import os
import scipy
import cv2
from tqdm import tqdm
import math
import ast
sns.set()

In [ ]:
df_train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
df_train

In [ ]:
def bbox_inv_iou(boxA, boxB):
    """Copied from: https://gist.github.com/meyerjo/dd3533edc97c81258898f60d8978eddc
    """
    xA, yA = max(boxA[0], boxB[0]), max(boxA[1], boxB[1])
    xB, yB = min(boxA[2], boxB[2]), min(boxA[3], boxB[3])
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))
    return 1. - (interArea / float(boxAArea + boxBArea - interArea))


def bbox_center_distance(boxA, boxB):
    cAx, cAy = (boxA[2] + boxA[0]) / 2., (boxA[3] + boxA[1]) / 2.
    cBx, cBy = (boxB[2] + boxB[0]) / 2., (boxB[3] + boxB[1]) / 2.
    return np.sqrt((cBx - cAx) ** 2 + (cBy - cAy) ** 2)


def bbox_center_abs_difference(boxA, boxB):
    cAx, cAy = (boxA[2] - boxA[0]) / 2., (boxA[3] - boxA[1]) / 2.
    cBx, cBy = (boxB[2] - boxB[0]) / 2., (boxB[3] - boxB[1]) / 2.
    return np.sqrt((cBx - cAx) ** 2 + (cBy - cAy) ** 2)


def bbox_center_rel_difference(boxA, boxB, multiplier=10.):
    cAx, cAy = (boxA[2] - boxA[0]) / 2., (boxA[3] - boxA[1]) / 2.
    cBx, cBy = (boxB[2] - boxB[0]) / 2., (boxB[3] - boxB[1]) / 2.
    ccA, ccB = np.sqrt(cAx**2 + cAy**2), np.sqrt(cBx**2 + cBy**2)
    return multiplier * max(ccA/(ccB + 1e-5), ccB/(ccA + 1e-5))


def find_unique_cots(sequence_df, dist_func=bbox_center_distance, dist_thresh=30.0, verbose=1):
    # Check that the df is valid
    unique_diff = np.unique(sequence_df.sequence_frame.diff())
    assert unique_diff[~np.isnan(unique_diff)] == np.ones(shape=(1, ))
    
    prev_bboxes_with_uuids = None
    annots_with_cots_ids, min_intra_distances = [], []
    
    # Some stats
    all_cots_ids, max_inter_distances, cots_per_frame = set(), [], []
    
    def create_new_cots_uuid():
        new_cots_id = uuid.uuid4().hex
        all_cots_ids.add(new_cots_id)
        return new_cots_id
    
    for idx, row in sequence_df.iterrows():
        raw_annots = ast.literal_eval(row.annotations)
        bboxes = [
            [s['x'], s['y'], s['x'] + s['width'], s['y'] + s['height']]
            for s in raw_annots]
        cots_per_frame.append(len(bboxes))
        
        # For frames with no annotation, reset everything.
        if len(bboxes) == 0:
            prev_bboxes_with_uuids = None
            annots_with_cots_ids.append('[]')
            min_intra_distances.append(0)
            max_inter_distances.append(0)
            continue
        
        # Intra-frame ious
        intra_distances = np.zeros(shape=(len(bboxes), len(bboxes)))
        for i, bbox_0 in enumerate(bboxes):
            for j, bbox_1 in enumerate(bboxes):
                if j == i:
                    intra_distances[i, j] = 1000000.
                else:
                    intra_distances[i, j] = dist_func(bbox_0, bbox_1)

        # Tracking COTS bounding-boxes and assign UUID to each COTS
        if prev_bboxes_with_uuids is None:
            prev_bboxes_with_uuids = [{
                'bbox': bbox,
                'cid': create_new_cots_uuid(),
                } for bbox in bboxes]
            max_inter_distances.append(0)
        else:
            # Calculate inter-frame IOUs
            distances = np.zeros(shape=(len(prev_bboxes_with_uuids), len(bboxes)))
            for i, bbox_with_uuid_0 in enumerate(prev_bboxes_with_uuids):
                for j, bbox in enumerate(bboxes):
                    distances[i, j] = dist_func(bbox_with_uuid_0['bbox'], bbox)
                    if distances[i, j] > dist_thresh:
                        distances[i, j] = 1000000.
            max_inter_distances.append(distances.max())
            
            row_ids, col_ids = scipy.optimize.linear_sum_assignment(distances)
            curr_bboxes_with_uuids, curr_matched_ids = [], []
            for prev_id, curr_id in zip(row_ids, col_ids):
                if distances[prev_id, curr_id] <= dist_thresh:
                    curr_matched_ids.append(curr_id)
                    curr_bboxes_with_uuids.append({
                        'bbox': bboxes[curr_id],
                        'cid': prev_bboxes_with_uuids[prev_id]['cid'],
                        'prev_bbox': prev_bboxes_with_uuids[prev_id]['bbox']
                    })
            for curr_id in range(len(bboxes)):
                if curr_id not in curr_matched_ids:
                    curr_bboxes_with_uuids.append({
                        'bbox': bboxes[curr_id],
                        'cid': create_new_cots_uuid(),
                    })
            
            # Prepare for next iteration
            prev_bboxes_with_uuids = curr_bboxes_with_uuids
        
        # Append calculated info
        min_intra_distances.append(intra_distances.min())
        annots_with_cots_ids.append(repr([
            {
                'cots_id': bb_w_id['cid'],
                'x': bb_w_id['bbox'][0],
                'y': bb_w_id['bbox'][1],
                'width': bb_w_id['bbox'][2] - bb_w_id['bbox'][0],
                'height': bb_w_id['bbox'][3] - bb_w_id['bbox'][1],
                'prev_bbox': bb_w_id.get('prev_bbox', None)
            }
            for bb_w_id in prev_bboxes_with_uuids]))

    sequence_df['min_intra_dist'] = min_intra_distances
    sequence_df['annots_with_cots_id'] = annots_with_cots_ids
    
    stats = {
        'unique_cots': len(all_cots_ids),
        'max_inter_frame_dist': np.max(max_inter_distances),
        'max_cots_per_frame': np.max(cots_per_frame),
    }
    return sequence_df, stats

In [ ]:
test_sequence_id = np.unique(df_train.sequence)[2]
print(test_sequence_id)
test_sequence_df = df_train[df_train.sequence == test_sequence_id]
test_sequence_df = test_sequence_df.sort_values(by='sequence_frame')
test_sequence_df = test_sequence_df.head(500)
test_sequence_df

In [ ]:
seq_df_with_cots_ids, stats = find_unique_cots(
    test_sequence_df,
    dist_func=lambda boxA, boxB: bbox_center_distance(boxA, boxB), # + bbox_center_rel_difference(boxA, boxB, multiplier=20.),
    dist_thresh=50.0)

print(f'Summary:')
for k, v in stats.items():
    print(f'  - {k}: {v}')

seq_df_with_cots_ids

In [ ]:
best_idx, best_row, most_cots = None, None, 0
for idx, row in seq_df_with_cots_ids.iterrows():
    raw_annots = ast.literal_eval(row.annots_with_cots_id)
    if len(raw_annots) > most_cots:
        best_idx, best_row, most_cots = idx, row, len(raw_annots)

print(best_idx, most_cots)
print(best_row)
print('\n'.join(str(s) for s in ast.literal_eval(best_row.annots_with_cots_id)))

In [ ]:
def load_image(video_id, video_frame, image_dir):
    img_path = f'{image_dir}/video_{video_id}/{video_frame}.jpg'
    assert os.path.exists(img_path), f'{img_path} does not exist.'
    img = cv2.imread(img_path)
    return img


def load_image_with_annotations(row, image_dir):
    video_id = row.video_id
    video_frame = row.video_frame
    annotaitons_str = row.annots_with_cots_id

    img = load_image(video_id, video_frame, image_dir)
    img_h, img_w = img.shape[:2]
    annotations = ast.literal_eval(annotaitons_str)
    palette = (np.array(sns.color_palette("hls", 16)) * 255).astype(np.int).tolist()
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    txt_anno = f'vid: {row.video_id} | seq: {row.sequence} | vid frm: {row.video_frame} | ' \
               f'seq frm: {row.sequence_frame} | cots: {len(annotations)}'
    cv2.putText(img, txt_anno, (10,25), font, 0.7, (30, 30, 30), 2)

    if len(annotations) > 0:
        for ann in annotations:
            main_box_thickness = 16
            instance_color = palette[int(ann['cots_id'][0], 16)][::-1]
            if ann['prev_bbox'] is not None:
                pbb = ann['prev_bbox']
                c_cx, c_cy = ann['x'] + ann['width'] // 2, ann['y'] + ann['height'] // 2
                p_cx, p_cy = (pbb[0] + pbb[2]) // 2, (pbb[1] + pbb[3]) // 2
                cv2.line(img, (p_cx, p_cy), (c_cx, c_cy), list(instance_color), thickness=8,)
                main_box_thickness = 4
            cv2.rectangle(img, (ann['x'], ann['y']),
                (ann['x'] + ann['width'], ann['y'] + ann['height']),
                list(instance_color), thickness=main_box_thickness,)
    return img

#test
video_id = best_row.video_id
video_frame = best_row.video_frame
annotations_str = best_row.annots_with_cots_id
image_dir = '../input/tensorflow-great-barrier-reef/train_images'
img = load_image_with_annotations(best_row, image_dir)

plt.figure(figsize=(15, 10))
plt.imshow(img[:, :, ::-1])
plt.axis('off')

In [ ]:
from tqdm.auto import tqdm
import subprocess

def make_video(df, video_name, image_dir):
    # partly borrowed from https://github.com/RobMulla/helmet-assignment/blob/main/helmet_assignment/video.py
    fps = 15 # don't know exact value
    width = 1280
    height = 720
    save_path = f'{video_name}.mp4'
    tmp_path = "tmp_" + save_path
    output_video = cv2.VideoWriter(tmp_path, cv2.VideoWriter_fourcc(*"MP4V"), fps, (width, height))
    
    video_df = df
    for _, row in tqdm(video_df.iterrows(), total=len(video_df)):
        img = load_image_with_annotations(row, image_dir)
        output_video.write(img)
    
    output_video.release()
    # Not all browsers support the codec, we will re-load the file at tmp_output_path
    # and convert to a codec that is more broadly readable using ffmpeg
    if os.path.exists(save_path):
        os.remove(save_path)
    subprocess.run(
        ["ffmpeg", "-i", tmp_path, "-crf", "18", "-preset", "veryfast", "-vcodec", "libx264", save_path]
    )
    os.remove(tmp_path)


make_video(seq_df_with_cots_ids, 'test_video', image_dir)

In [ ]:
from IPython.display import Video, display
Video('test_video.mp4')

# Generate videos for each sequence

In [ ]:
additional_columns_by_seqid = []

for sequence_id in np.unique(df_train.sequence):
    sequence_df = df_train[df_train.sequence == sequence_id]
    sequence_df = sequence_df.sort_values(by='sequence_frame')
    
    seq_df_with_cots_ids, stats = find_unique_cots(
        sequence_df,
        dist_func=lambda boxA, boxB: bbox_center_distance(boxA, boxB),
        dist_thresh=50.0)

    print(f'Sequence {sequence_id:05d} summary:')
    for k, v in stats.items():
        print(f'  - {k}: {v}')
    
    additional_columns_by_seqid.append(seq_df_with_cots_ids.loc[:, ['annots_with_cots_id', 'min_intra_dist']])
    make_video(seq_df_with_cots_ids, f'sequence_{sequence_id:05d}', image_dir)

In [ ]:
df_train.join(pd.concat(additional_columns_by_seqid)).to_csv('train_with_cots_ids.csv')